# PV Degradation Tools Tutorial - 2
### Degradation and Solder Fatigue
***
Matthew Brown

2023.06.02
***

**Requirements**:
- compatible weather file (PSM3, TMY3, EPW)
- Accelerated testing chamber parameters
    - chamber irradiance [W/m^2]
    - chamber temperature [C]
    - chamber humidity [%]
- Activation energies for test material [kJ/mol]

**Objectives**:
1. Read in the weather data
2. Calculate VantHoff degradation acceleration factor
3. Calculate Arrhenius degradation acceleration factor
4. Solder Fatigue

## VantHoff

In [ ]:
# chamber irradiance (W/m^2)
I_chamber = 1000 
# chamber temperature (C)
temp_chamber = 60

# calculate the cell temperature with pvlib (see documentation for help)
cell_params = pvlib.temperature.TEMPERATURE_MODEL_PARAMETERS['sapm']['open_rack_glass_polymer']

temp_cell = pvlib.temperature.sapm_cell(poa_global=poa_global, temp_air=data['Temperature'],
                                        wind_speed=data['Wind Speed'], **cell_params)

# calculate the VantHoff Acceleration factor
vantHoff_deg = pvdeg.Degradation.vantHoff_deg(I_chamber=I_chamber, poa_global=poa_global,
                                            temp_cell=temp_cell, temp_chamber=temp_chamber)

# calculate the VantHoff weighted irradiance
irr_weighted_avg_v = pvdeg.Degradation.IwaVantHoff(poa_global=poa_global, temp_cell=temp_cell)

## Arrhenius

In [ ]:
# relative humidity within chamber (%)
rh_chamber = 15
# arrhenius activation energy (kj/mol)
Ea = 40

# calculate the module temperature with pvlib (see documentation for help)
temp_module = pvlib.temperature.sapm_module(poa_global=poa_global, temp_air=data['Temperature'],
                                            wind_speed=data['Wind Speed'], a=cell_params['a'],
                                            b=cell_params['b'])

rh_surface = pvdeg.StressFactors.rh_surface_outside(rh_ambient=data['Relative Humidity'],
                                                        temp_ambient=data['Temperature'],
                                                        temp_module=temp_module)

arrhenius_deg = pvdeg.Degradation.arrhenius_deg(I_chamber=I_chamber, rh_chamber=rh_chamber,
                                              rh_outdoor=rh_surface, poa_global=poa_global,
                                              temp_chamber=temp_chamber, temp_cell=temp_cell,
                                              Ea=Ea)

irr_weighted_avg_a = pvdeg.Degradation.IwaArrhenius(poa_global=poa_global, 
                                                  rh_outdoor=data['Relative Humidity'],
                                                  temp_cell=temp_cell, Ea=Ea)